In [6]:
import mlflow
import pandas as pd
import numpy as np
import json
import requests

### Testing The API - Online Prediction EndPoint

In [7]:
# The data that will be sent in the request body
player = {
  "gp": 0.0,
  "min": 0.0,
  "pts": 0.0,
  "fgm": 0.0,
  "fga": 0.0,
  "fgpercent": 0.0,
  "treepmade": 0.0,
  "treepa": 0.0,
  "treeppercent": 0.0,
  "ftm": 0.0,
  "fta": 0.0,
  "ftpercent": 0.0,
  "oreb": 0.0,
  "dreb": 0.0,
  "reb": 0.0,
  "ast": 0.0,
  "stl": 0.0,
  "blk": 0.0,
  "tov": 0.0,
  "model_name": "nba-investment-model-f1"
}

# The url of the endpoint that the request will be sent to
url = "http://20.224.70.229:4100/online_predict"

# The headers of the request, in this case it's setting the accept and Content-Type
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# Send the request
response = requests.post(url, headers=headers, json=player)

# Print the json data from the response
print(response.json())

{'prediction': 0, 'status': 200, 'message': 'Bad investment', 'model_name': 'nba-investment-model-f1'}


### Testing The API - Batch Prediction EndPoint

In [8]:
# Specifying the endpoint url
url = "http://20.224.70.229:4100/batch_predict?model_name=nba-investment-model-f1"


# Specifying the file to be sent
files=[
  ('file',('nba_logreg_preprocessed.csv',open('/home/abdoss/mlops-project-nba--investment/data/nba_logreg_preprocessed.csv','rb'),'text/csv'))
]

# Sending the request
response = requests.request("POST", url, headers={}, data={}, files=files)

# Printing the response
print(response.json())

{'file_name': 'predictions_3c1715a4-b10f-4b20-af4a-15e065ddb8e2.csv', 'status': 200, 'model_name': 'nba-investment-model-f1'}


In [9]:
import requests
import random

# The URL of the deployed model
URL = "http://20.224.70.229:4100/online_predict"

# The headers of the request, in this case it's setting the accept and Content-Type
headers = {"accept": "application/json", "Content-Type": "application/json"}


def generate_random_data(num_points):
    data = []
    for _ in range(num_points):
        player = {
            "gp": random.uniform(1, 200),
            "min": random.uniform(1, 200),
            "pts": random.uniform(1, 200),
            "fgm": random.uniform(1, 200),
            "fga": random.uniform(1, 200),
            "fgpercent": random.uniform(1, 200),
            "treepmade": random.uniform(1, 200),
            "treepa": random.uniform(1, 200),
            "treeppercent": random.uniform(1, 200),
            "ftm": random.uniform(1, 200),
            "fta": random.uniform(1, 200),
            "ftpercent": random.uniform(1, 200),
            "oreb": random.uniform(1, 200),
            "dreb": random.uniform(1, 200),
            "reb": random.uniform(1, 200),
            "ast": random.uniform(1, 200),
            "stl": random.uniform(1, 200),
            "blk": random.uniform(1, 200),
            "tov": random.uniform(1, 200),
            "model_name": "nba-investment-model-f1"
        }
        data.append(player)
    return data

players = generate_random_data(100)
print(players[0])

for player in players:
    # Send the request
    response = requests.post(URL, headers=headers, json=player)
    prediction = response.json()
    print("the prediction is: ", prediction)



{'gp': 171.91718829572645, 'min': 40.64089254049795, 'pts': 83.83493486961905, 'fgm': 100.36375578411555, 'fga': 103.78396242275646, 'fgpercent': 173.31013990898865, 'treepmade': 74.9116066427034, 'treepa': 169.62238329821162, 'treeppercent': 176.91789135892196, 'ftm': 136.38384675512998, 'fta': 103.61586795690431, 'ftpercent': 92.33231654617845, 'oreb': 38.855439267574916, 'dreb': 13.94200741475321, 'reb': 87.37115287988746, 'ast': 139.30968402662722, 'stl': 53.56435718355318, 'blk': 189.2707109780575, 'tov': 36.599994535735576, 'model_name': 'nba-investment-model-f1'}
the prediction is:  {'prediction': 1, 'status': 200, 'message': 'Good investment', 'model_name': 'nba-investment-model-f1'}
the prediction is:  {'prediction': 1, 'status': 200, 'message': 'Good investment', 'model_name': 'nba-investment-model-f1'}
the prediction is:  {'prediction': 1, 'status': 200, 'message': 'Good investment', 'model_name': 'nba-investment-model-f1'}
the prediction is:  {'prediction': 1, 'status': 200